In [ ]:
import time
import talib
import numpy as np
import pandas as pd
import requests
from binance.client import Client
from binance.enums import *
from binance.exceptions import BinanceAPIException
from apscheduler.schedulers.blocking import BlockingScheduler

# ✅ Enter Your Binance API Key and Secret Here
API_KEY = "MrittnLSwQXCOMEEF1By9jwKeYY5pZnXntcqfCPBUQpFv3LkPI5Zc1JXbynrvGcN"
API_SECRET = "NcSVtO5U5B3362126CEOD4JlSjRMFCDFwPOuUUgetSt40mUW1mJoDXJDylcSryk9"

# ✅ Check if API Keys Are Present
if not API_KEY or not API_SECRET:
    print("❌ Binance API Keys are missing! Please enter them in the script.")
    exit()

# ✅ Initialize Binance Client with Error Handling
try:
    client = Client(API_KEY, API_SECRET)
    client.ping()  # Ensure API is reachable
    print("✅ Binance API Connection Successful!")
except BinanceAPIException as e:
    print(f"❌ Binance API Error: {e}")
    exit()
except requests.exceptions.RequestException as e:
    print(f"❌ Network Error: {e}")
    exit()

# ✅ Function to Safely Call Binance API with Retry Mechanism
def safe_binance_call(func, *args, **kwargs):
    max_retries = 5
    retry_delay = 5  # Wait 5 seconds before retrying

    for attempt in range(max_retries):
        try:
            return func(*args, **kwargs)
        except BinanceAPIException as e:
            if "502 Bad Gateway" in str(e):
                print(f"⚠️ Binance 502 Error (Attempt {attempt+1}/{max_retries}) - Retrying in {retry_delay}s...")
                time.sleep(retry_delay)
            else:
                print(f"❌ Binance API Exception: {e}")
                return None
        except requests.exceptions.RequestException as e:
            print(f"❌ Network Error: {e}")
            return None
    print("❌ Binance API Failed After Multiple Attempts!")
    return None

# ✅ Function to Fetch Historical Data
def get_historical_data(symbol, interval, lookback):
    klines = safe_binance_call(client.get_klines, symbol=symbol, interval=interval, limit=lookback)
    if not klines:
        return None
    
    df = pd.DataFrame(klines, columns=['time', 'open', 'high', 'low', 'close', 'volume', 
                                       'close_time', 'quote_asset_volume', 'num_trades', 
                                       'taker_buy_base_vol', 'taker_buy_quote_vol', 'ignore'])
    
    df[['open', 'high', 'low', 'close']] = df[['open', 'high', 'low', 'close']].astype(float)
    return df

# ✅ Function to Identify Candlestick Patterns
def identify_patterns(df):
    patterns = {
        'Hammer': talib.CDLHAMMER(df['open'], df['high'], df['low'], df['close']),
        'Engulfing': talib.CDLENGULFING(df['open'], df['high'], df['low'], df['close']),
        'Morning Star': talib.CDLMORNINGSTAR(df['open'], df['high'], df['low'], df['close']),
        'Evening Star': talib.CDLEVENINGSTAR(df['open'], df['high'], df['low'], df['close']),
        'Shooting Star': talib.CDLSHOOTINGSTAR(df['open'], df['high'], df['low'], df['close']),
        'Doji': talib.CDLDOJI(df['open'], df['high'], df['low'], df['close']),
        'Three White Soldiers': talib.CDL3WHITESOLDIERS(df['open'], df['high'], df['low'], df['close']),
        'Three Black Crows': talib.CDL3BLACKCROWS(df['open'], df['high'], df['low'], df['close']),
        'Bullish Harami': talib.CDLHARAMI(df['open'], df['high'], df['low'], df['close']),
        'Bearish Harami': talib.CDLHARAMICROSS(df['open'], df['high'], df['low'], df['close'])
    }
    
    signals = [key for key, val in patterns.items() if val.iloc[-1] != 0]
    return signals

# ✅ Function to Generate Trading Signals
def generate_signal(symbol):
    df = get_historical_data(symbol, Client.KLINE_INTERVAL_15MINUTE, 50)
    if df is None:
        return None, None

    patterns = identify_patterns(df)

    if patterns:
        last_close = df['close'].iloc[-1]
        
        if any(p in patterns for p in ['Hammer', 'Morning Star', 'Engulfing', 'Three White Soldiers', 'Bullish Harami']):
            return "LONG", last_close
        elif any(p in patterns for p in ['Shooting Star', 'Evening Star', 'Three Black Crows', 'Bearish Harami']):
            return "SHORT", last_close

    return None, None

# ✅ Function to Execute Trades
def execute_trade(symbol):
    signal, entry_price = generate_signal(symbol)
    if signal:
        print(f"\n📊 {symbol} - {signal} Signal Detected at {entry_price}")
        
        stop_loss = entry_price * (0.97 if signal == "LONG" else 1.03)
        tp1 = entry_price * (1.02 if signal == "LONG" else 0.98)
        tp2 = entry_price * (1.04 if signal == "LONG" else 0.96)
        tp3 = entry_price * (1.06 if signal == "LONG" else 0.94)
        
        print(f"Entry: {entry_price}, Stop Loss: {stop_loss}, TP1: {tp1}, TP2: {tp2}, TP3: {tp3}")
        
        order_type = ORDER_TYPE_MARKET
        side = SIDE_BUY if signal == "LONG" else SIDE_SELL
        
        try:
            order = safe_binance_call(client.create_order,
                symbol=symbol,
                side=side,
                type=order_type,
                quantity=0.01  # Modify lot size
            )
            if order:
                print(f"✅ Trade Executed: {order}")
        except Exception as e:
            print(f"⚠️ Trade Execution Error: {e}")

# ✅ Function to Scan and Trade the Top 100 USDT Pairs
def trading_bot():
    usdt_pairs = [s['symbol'] for s in safe_binance_call(client.get_exchange_info)['symbols'] if s['symbol'].endswith("USDT")]
    
    if not usdt_pairs:
        print("❌ No USDT pairs found!")
        return
    
    for symbol in usdt_pairs[:100]:  # Check Top 100 USDT pairs
        execute_trade(symbol)
        time.sleep(2)

# ✅ Scheduler to Run the Bot Every 5 Minutes
scheduler = BlockingScheduler()
scheduler.add_job(trading_bot, 'interval', minutes=5)
print("🚀 Trading bot started... Press Ctrl+C to stop.")
scheduler.start()

✅ Binance API Connection Successful!
🚀 Trading bot started... Press Ctrl+C to stop.

📊 BTCUSDT - LONG Signal Detected at 90956.2
Entry: 90956.2, Stop Loss: 88227.514, TP1: 92775.324, TP2: 94594.448, TP3: 96413.572
❌ Binance API Exception: APIError(code=-2010): Account has insufficient balance for requested action.

📊 ETHUSDT - LONG Signal Detected at 2291.63
Entry: 2291.63, Stop Loss: 2222.8811, TP1: 2337.4626000000003, TP2: 2383.2952, TP3: 2429.1278
❌ Binance API Exception: APIError(code=-2010): Account has insufficient balance for requested action.

📊 IOTAUSDT - LONG Signal Detected at 0.2222
Entry: 0.2222, Stop Loss: 0.215534, TP1: 0.226644, TP2: 0.23108800000000002, TP3: 0.23553200000000002
❌ Binance API Exception: APIError(code=-1013): Filter failure: LOT_SIZE

📊 XLMUSDT - LONG Signal Detected at 0.3052
Entry: 0.3052, Stop Loss: 0.29604400000000003, TP1: 0.311304, TP2: 0.317408, TP3: 0.323512
❌ Binance API Exception: APIError(code=-1013): Filter failure: LOT_SIZE

📊 ONTUSDT - LONG

Execution of job "trading_bot (trigger: interval[0:05:00], next run at: 2025-03-06 15:48:00 IST)" skipped: maximum number of running instances reached (1)



📊 NEOUSDT - SHORT Signal Detected at 9.24
Entry: 9.24, Stop Loss: 9.5172, TP1: 9.0552, TP2: 8.8704, TP3: 8.685599999999999
❌ Binance API Exception: APIError(code=-1013): Filter failure: NOTIONAL

📊 TUSDUSDT - LONG Signal Detected at 0.9979
Entry: 0.9979, Stop Loss: 0.967963, TP1: 1.017858, TP2: 1.037816, TP3: 1.057774
❌ Binance API Exception: APIError(code=-1013): Filter failure: LOT_SIZE

📊 BCHSVUSDT - LONG Signal Detected at 58.9
Entry: 58.9, Stop Loss: 57.132999999999996, TP1: 60.078, TP2: 61.256, TP3: 62.434000000000005
❌ Binance API Exception: APIError(code=-1013): Market is closed.

📊 XMRUSDT - LONG Signal Detected at 118.7
Entry: 118.7, Stop Loss: 115.139, TP1: 121.074, TP2: 123.44800000000001, TP3: 125.822
❌ Binance API Exception: APIError(code=-1013): Market is closed.

📊 ANKRUSDT - SHORT Signal Detected at 0.01959
Entry: 0.01959, Stop Loss: 0.0201777, TP1: 0.0191982, TP2: 0.018806399999999997, TP3: 0.0184146
❌ Binance API Exception: APIError(code=-1013): Filter failure: LOT_